In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
from typing import List
import re
import string

In [2]:
df = pd.read_csv('data_classification/train.csv')

titles = df['title'].to_list()
descriptions = df['description'].to_list()
labels = df['label'].to_list()

In [3]:
def remove_punctuations(text: List[str]) -> List[str]:
    """To remove all the punctuations present in the text.Input the text column"""
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)


def tokenize_sentence(sentence: str) -> List[str]:
    """Tokenizes a sentence into words

    Args:
        sentence (str): Input sentence

    Returns:
        List[str]: List of words (tokens)
    """
    sentence = remove_punctuations(sentence)
    sentence = re.sub(r"[^A-Za-z0-9(),.!?\'`\-\"]", " ", sentence)
    sentence = re.sub(r"\'s", " 's", sentence)
    sentence = re.sub(r"\'ve", " 've", sentence)
    sentence = re.sub(r"n\'t", " n't", sentence)
    sentence = re.sub(r"\'re", " 're", sentence)
    sentence = re.sub(r"\'d", " 'd", sentence)
    sentence = re.sub(r"\'ll", " 'll", sentence)
    sentence = re.sub(r"\.", " . ", sentence)
    sentence = re.sub(r",", " , ", sentence)
    sentence = re.sub(r"!", " ! ", sentence)
    sentence = re.sub(r"\?", " ? ", sentence)
    sentence = re.sub(r"\(", " ( ", sentence)
    sentence = re.sub(r"\)", " ) ", sentence)
    sentence = re.sub(r"\-", " - ", sentence)
    sentence = re.sub(r"\"", ' " ', sentence)
    # We may have introduced double spaces, so collapse these down
    sentence = re.sub(r"\s{2,}", " ", sentence)
    sentence = sentence.lower()
    return list(filter(lambda x: len(x) > 0, sentence.split(" ")))


def tokenize(text: List[str]) -> List[List[str]]:
    """Tokenizes a list of sentences into words

    Args:
        text (List[str]): List of sentences

    Returns:
        List[List[str]]: List of list of words (tokens)
    """
    return [tokenize_sentence(sentence) for sentence in text]

In [4]:
titles = tokenize(titles)
descriptions = tokenize(descriptions)

In [5]:
texts =  [title + description for title, description in zip(titles, descriptions)]

In [6]:
print(texts[:10])

[['wall', 'st', 'bears', 'claw', 'back', 'into', 'the', 'black', 'reuters', 'reuters', 'shortsellers', 'wall', 'streets', 'dwindlingband', 'of', 'ultracynics', 'are', 'seeing', 'green', 'again'], ['carlyle', 'looks', 'toward', 'commercial', 'aerospace', 'reuters', 'reuters', 'private', 'investment', 'firm', 'carlyle', 'groupwhich', 'has', 'a', 'reputation', 'for', 'making', 'welltimed', 'and', 'occasionallycontroversial', 'plays', 'in', 'the', 'defense', 'industry', 'has', 'quietly', 'placedits', 'bets', 'on', 'another', 'part', 'of', 'the', 'market'], ['oil', 'and', 'economy', 'cloud', 'stocks', 'outlook', 'reuters', 'reuters', 'soaring', 'crude', 'prices', 'plus', 'worriesabout', 'the', 'economy', 'and', 'the', 'outlook', 'for', 'earnings', 'are', 'expected', 'tohang', 'over', 'the', 'stock', 'market', 'next', 'week', 'during', 'the', 'depth', 'of', 'thesummer', 'doldrums'], ['iraq', 'halts', 'oil', 'exports', 'from', 'main', 'southern', 'pipeline', 'reuters', 'reuters', 'authorities

In [7]:
# create word2idx dictionary

word2idx = {}
idx = 1
for text in texts:
    for word in text:
        if word not in word2idx:
            word2idx[word] = idx
            idx += 1

In [8]:
text_ints: List[List[int]] = []
for text in texts:
    text_ints.append([word2idx[word] for word in text])

In [9]:
texts = text_ints

In [10]:
encoded_labels = [[1 if label == i + 1 else 0 for i in range(4)] for label in labels]

In [12]:
# pad texts so that each text has the same length
def pad_features(texts, seq_length):
    features = []
    for text in texts:
        if len(text) >= seq_length:
            features.append(text[:seq_length])
        else:
            features.append([0] * (seq_length - len(text)) + text)
    return features

In [13]:
# find longest text
text_lens = Counter([len(text) for text in texts])
print("Zero-length texts: {}".format(text_lens[0]))
print("Maximum text length: {}".format(max(text_lens)))

Zero-length texts: 0
Maximum text length: 177


In [14]:
seq_len = 200
features = pad_features(texts, seq_len)

In [15]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

train_x = np.array(train_x)
train_y = np.array(train_y)
val_x = np.array(val_x)
val_y = np.array(val_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(96000, 200) 
Validation set: 	(12000, 200) 
Test set: 		(12000, 200)


In [11]:
print(encoded_labels[:10])

[[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]]


In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 64

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [17]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([64, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,  1438,  1439,  1371],
        [    0,     0,     0,  ...,    29,  3250,  2098],
        [    0,     0,     0,  ...,   213,     7, 13461],
        ...,
        [    0,     0,     0,  ...,    31,     7,   431],
        [    0,     0,     0,  ...,   896,  2392,  2098],
        [    0,     0,     0,  ...,  2563,  6166, 70235]])

Sample label size:  torch.Size([64, 4])
Sample label: 
 tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 1, 0,

In [18]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [22]:
import torch.nn as nn

class ClassificationRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(ClassificationRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Softmax(dim=1)
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out[:, -1, :] # getting the last time step output
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [23]:
# Instantiate the model w/ hyperparams
vocab_size = len(word2idx)+1 # +1 for the 0 padding + our word tokens
output_size = 4
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = ClassificationRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

ClassificationRNN(
  (embedding): Embedding(102170, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (sig): Softmax(dim=1)
)


In [24]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [27]:
# training params

epochs = 2

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/2... Step: 100... Loss: 0.903415... Val Loss: 0.966732
Epoch: 1/2... Step: 200... Loss: 0.945033... Val Loss: 0.947369
Epoch: 1/2... Step: 300... Loss: 0.931886... Val Loss: 0.942802


KeyboardInterrupt: 

In [ ]:
# Make test loop for multiclass classification
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data

for inputs, labels in test_loader:
    
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
    
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
    
        # get predicted outputs
        output, h = net(inputs, h)
    
        # calculate loss
        test_loss = criterion(output.squeeze(), labels.float())
        test_losses.append(test_loss.item())
    
        # convert output probabilities to predicted class (0 or 1)
        prediction = torch.argmax(output, 1)
    
        # compare predictions to true label
        true_label = torch.argmax(labels, 1)

        correct_tensor = prediction.eq(true_label.data.view_as(prediction))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        num_correct += np.sum(correct)

# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)

print("Test accuracy: {:.3f}".format(test_acc))

